In [1]:
from Algorithms import MEAStar
import networkx as nx
import pickle
from typing import List, Set, Tuple, Dict
import numpy as np

In [2]:
def import_graph_pickle(filename='rack_network.gpickle'):
    """
    Import graph from pickle format.
    
    Args:
        filename: Input filename
    
    Returns:
        NetworkX graph
    """
    with open(filename, 'rb') as f:
        G = pickle.load(f)
    print(f"Graph loaded from {filename}")
    print(f"Nodes: {G.number_of_nodes()}, Edges: {G.number_of_edges()}")
    return G

In [3]:
G_combined = import_graph_pickle("Intermediate Outputs/3d connected network.gpickle")

Graph loaded from Intermediate Outputs/3d connected network.gpickle
Nodes: 66, Edges: 444


In [4]:
mea_star = MEAStar(G_combined)
path = mea_star.find_path(0, 1)

In [5]:
if path:
    print(f"Path found: {path}")
    print(f"Path length: {len(path)} nodes")
else:
    print("No path found")

Path found: ([0, 1], 254.7501000055775)
Path length: 2 nodes


## Create Cost Matrix for ACO

In [6]:
def create_cost_matrix(mea_star_instance, nodes: List) -> Tuple[np.ndarray, List, Dict]:
    """
    Create a cost matrix for a given set of nodes using MEA* algorithm.
    Nodes 0 and 100 are automatically included regardless of input.
    
    Args:
        mea_star_instance: Instance of MEAStar class with a graph loaded
        nodes: List of node IDs to include in the matrix
        
    Returns:
        Tuple containing:
        - cost_matrix: numpy array of shape (n, n) with costs between nodes
        - node_list: ordered list of nodes (matches matrix indices)
        - node_to_index: dictionary mapping node ID to matrix index
    """
    # Ensure nodes 0 and 100 are included
    node_set = set(nodes)
    node_set.add(0)
    node_set.add(100)
    
    # Convert to sorted list for consistent ordering
    node_list = sorted(list(node_set))
    
    # Create mapping from node ID to matrix index
    node_to_index = {node: idx for idx, node in enumerate(node_list)}
    
    # Get number of nodes
    n = len(node_list)
    
    # Initialize cost matrix with infinity
    cost_matrix = np.full((n, n), np.inf)
    
    # Set diagonal to 0 (cost from node to itself)
    np.fill_diagonal(cost_matrix, 0)
    
    # Calculate costs between all pairs of nodes
    print(f"Calculating cost matrix for {n} nodes...")
    
    for i, start_node in enumerate(node_list):
        for j, end_node in enumerate(node_list):
            if i == j:
                continue  # Skip diagonal (already set to 0)
            
            # Find path and cost using MEA*
            path, cost = mea_star_instance.find_path(start_node, end_node)
            print(start_node, " - ", end_node, ' - ', cost)
            
            if path is not None:
                cost_matrix[i, j] = cost
            else:
                # If no path exists, keep as infinity
                cost_matrix[i, j] = np.inf
            
            # Optional: print progress
            if (i * n + j + 1) % 10 == 0:
                print(f"Progress: {i * n + j + 1}/{n * n} paths calculated")
    
    print("Cost matrix calculation complete!")
    
    return cost_matrix, node_list, node_to_index


def print_cost_matrix_info(cost_matrix: np.ndarray, node_list: List, node_to_index: Dict):
    """
    Print information about the cost matrix for verification.
    
    Args:
        cost_matrix: The cost matrix
        node_list: List of nodes
        node_to_index: Mapping from node to index
    """
    print(f"\nCost Matrix Shape: {cost_matrix.shape}")
    print(f"Nodes included: {node_list}")
    print(f"\nNode to Index Mapping:")
    for node, idx in sorted(node_to_index.items()):
        print(f"  Node {node} -> Index {idx}")
    
    print(f"\nSample costs:")
    print(f"  Cost from node 0 to node 100: {cost_matrix[node_to_index[0], node_to_index[100]]}")
    
    # Show nodes with no path (infinite cost)
    inf_count = np.isinf(cost_matrix).sum() - len(node_list)  # Exclude diagonal
    print(f"\nNumber of unreachable node pairs: {inf_count}")

In [7]:
# Create cost matrix for specific nodes (0 and 100 will be added automatically)
selected_nodes = [1, 5]
cost_matrix, node_list, node_to_index = create_cost_matrix(mea_star, selected_nodes)

# Print information
print_cost_matrix_info(cost_matrix, node_list, node_to_index)

# Example: Access specific costs
print(f"\nExample access:")
print(f"Cost from node 0 to node 5: {cost_matrix[node_to_index[0], node_to_index[5]]}")
print(f"Cost from node 1 to node 100: {cost_matrix[node_to_index[1], node_to_index[100]]}")

Calculating cost matrix for 4 nodes...
0  -  1  -  254.7501000055775
0  -  5  -  891.7957600608995
0  -  100  -  None
1  -  0  -  None
1  -  5  -  637.0456600553221
1  -  100  -  None
5  -  0  -  None
5  -  1  -  637.0456600553221
Progress: 10/16 paths calculated
5  -  100  -  None
100  -  0  -  None
100  -  1  -  327.53584286431396
100  -  5  -  964.5815029196359
Cost matrix calculation complete!

Cost Matrix Shape: (4, 4)
Nodes included: [0, 1, 5, 100]

Node to Index Mapping:
  Node 0 -> Index 0
  Node 1 -> Index 1
  Node 5 -> Index 2
  Node 100 -> Index 3

Sample costs:
  Cost from node 0 to node 100: inf

Number of unreachable node pairs: 2

Example access:
Cost from node 0 to node 5: 891.7957600608995
Cost from node 1 to node 100: inf


In [8]:
np.savetxt('Intermediate Outputs/data.csv', cost_matrix, delimiter=',')

In [10]:
node_to_index

{0: 0, 1: 1, 5: 2, 100: 3}